In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import scipy.io as sio
import xarray as xr

from mapper_functions import plot_global_tight

In [ ]:
ds = xr.open_dataset('DAv7_M36.inst3_1d_lndfcstana_Nt.20150901.nc4')

# List of the variables in the file
print(ds.data_vars)

lon = ds['lon']
lat = ds['lat']

n_tile = len(lat)

print('n_tile = ',n_tile)

plt.figure()
plt.scatter(lon, lat)
plt.xlabel('lon')
plt.ylabel('lat')
plt.show()

# Load the calculated values from the file
data = np.load('../test_data/AMS_24/DAv7_M36_ASCAT_type_2_fp_precip_increment_stats.npz' , allow_pickle=True)

std_sfmc_increment_fp = data['std_sfmc_increment']
std_rzmc_increment_fp = data['std_rzmc_increment']

ob_array = np.empty([n_tile, 3])
ob_array.fill(np.nan)
ob_array[:, 1] = lon
ob_array[:, 2] = lat
ob_array[:, 0] = std_sfmc_increment_fp

plot_global_tight(ob_array,False,'ASC_FP: StdDev of surface SM increment','($m^3 \, m^{-3}$)', 0, 0.01, cmap='Blues')

In [ ]:
# Open .mat files

data = sio.loadmat('../test_data/AMS_24/IVs/Rdiff_DAv7ASCt2_M36_FPprcp_minus_OLv7_M36_FPprcp.mat')

# Get the data from the .mat file which contains 3 variables, Rdiff_vector, lons, lats
Rdiff_vector = np.squeeze(data['Rdiff_vector'])
lons = np.squeeze(data['lons'])
lats = np.squeeze(data['lats'])

print('len(lons) = ', len(lons))
plt.figure()
plt.scatter(lons, lats)
plt.xlabel('lon')
plt.ylabel('lat')
plt.show()

m36_lons = np.unique(lons)
m36_lats = np.unique(lats)

ob_array2 = np.empty([len(Rdiff_vector),3])
ob_array2.fill(np.nan)
ob_array2[:,1] = lons
ob_array2[:,2] = lats
ob_array2[:,0] = Rdiff_vector

# Plot the data
plot_global_tight(ob_array2,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

print("Mean:" + str(np.nanmean(Rdiff_vector)))

In [ ]:
import pandas as pd

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(ob_array2, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
ob_array3 = df3[['value2', 'lon', 'lat']].values

# Replace NaN values with np.nan
ob_array3[np.isnan(ob_array3[:, 0]), 0] = np.nan

plot_global_tight(ob_array3,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.8, 0.8)


plt.figure()
plt.scatter(ob_array3[:,1], ob_array3[:,2])
plt.xlabel('lon')
plt.ylabel('lat')
plt.show()

In [ ]:
# make a scatter plot of the two variables
plt.figure()
plt.scatter(ob_array[:,0], ob_array3[:,0])
plt.xlabel('StdDev of surface SM increment')
plt.ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.show()


In [ ]:
# Plot a probability density function of the two variables
plt.figure()
plt.hist(ob_array3[:,0], bins=50, alpha=0.5, label='ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.legend(loc='upper right')
plt.show()

# Print the fraction of the values that are below 0
print("Fraction of values below 0: " + str(np.sum(ob_array3[:,0] < 0) / len(ob_array3[:,0])))

In [ ]:
# Load the calculated values from the file which was save as:
# np.savez(file_name_start, lat=lat, lon=lon, ssom=ssom, tpcx=tpcx, domo=domo, smpf=smpf, smcf=smcf, alfr=alfr, iwfr=iwfr, snoc=snoc, flsf=flsf)
data = np.load('M01-ASCA-ASCSMO02-NA-5.0-2015080.npz' , allow_pickle=True)

# List the variables in the file
print(data.files)

qc_lats = np.squeeze(data['lat'])
qc_lons = np.squeeze(data['lon'])
ssom = np.squeeze(data['ssom'])
smpf = np.squeeze(data['smpf'])
smcf = np.squeeze(data['smcf'])

mask = (smpf == 0) # & (smcf == 0)
ob_array4 = np.empty([np.sum(mask), 3])
ob_array4.fill(np.nan)
ob_array4[:, 1] = qc_lons[mask]
ob_array4[:, 2] = qc_lats[mask]
ob_array4[:, 0] = smcf[mask]

plot_global_tight(ob_array4,False,'smcf','QC', 0, 8, cmap='viridis')


# Scatter plot of qc_lons[mask] and qc_lats[mask]
plt.figure()
plt.scatter(qc_lons[mask], qc_lats[mask])
plt.xlabel('lon')
plt.ylabel('lat')
plt.show()

In [ ]:
mask = (smpf == 0) & ((smcf == 0) | (smcf == 4))
qc_lons2 = qc_lons[mask]
qc_lats2 = qc_lats[mask]
smcf2 = smcf[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(qc_lons2))
closest_m36_lats = np.empty(len(qc_lats2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))

for i in range(len(qc_lons2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - qc_lons2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - qc_lats2[i]))]

m36_lons = np.unique( ob_array3[:,1])
m36_lats = np.unique(ob_array3[:,2])

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))

for i in range(len(qc_lons2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - qc_lons2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - qc_lats2[i]))]    


In [ ]:
from scipy import stats

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

# Find the smcf values for each unique pair
unique_smcf = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_smcf[i] = stats.mode(smcf2[mask])[0][0]

# Plot the data
ob_array5 = np.column_stack((unique_smcf, unique_pairs[:, 0], unique_pairs[:, 1]))
plot_global_tight(ob_array5,False,'smcf','QC', 0, 8, cmap='viridis')




In [ ]:
# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(ob_array5, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
ob_array6 = df3[['value2', 'lon', 'lat']].values

# Replace NaN values with np.nan
ob_array6[np.isnan(ob_array6[:, 0]), 0] = np.nan

plot_global_tight(ob_array6,False,'smcf','QC', 0, 8, cmap='viridis')


plt.figure()
plt.scatter(ob_array6[:,0], ob_array3[:,0])
plt.xlabel('smcf')
plt.ylabel('delta')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

qc0_values = ob_array6[:, 0] == 0
qc4_values = ob_array6[:, 0] == 4
delta_values = ob_array3[:, 0]

# Plot a probability density function of the two variables
plt.figure()
plt.title('QC flags')
plt.hist(delta_values[qc0_values], bins=50, alpha=0.5, label='QC=0')
plt.hist(delta_values[qc4_values], bins=50, alpha=0.5, label='QC=4')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = 'QC_histogram.png'
plt.savefig(filename, dpi=300)
plt.show()


In [ ]:
# np.savez('DAv7_M36_ASCAT_type_2_obsfcstana_stats.npz', unique_tilenum=unique_tilenum, num_obs=num_obs, mean_obs_obs=mean_obs_obs, std_obs_obs=std_obs_obs, mean_obs_fcst=mean_obs_fcst, mean_obs_ana=mean_obs_ana, mean_obs_minus_fcst=mean_obs_minus_fcst, mean_obs_minus_ana=mean_obs_minus_ana, std_obs_minus_fcst=std_obs_minus_fcst, std_obs_minus_ana=std_obs_minus_ana, lon_tilenum=lon_tilenum, lat_tilenum=lat_tilenum )
data = np.load('../test_data/DAv7_M36_ASCAT_type_2_obsfcstana_stats.npz', allow_pickle=True)

# List the variables in the file
print(data.files)

unique_tilenum = data['unique_tilenum']
num_obs = data['num_obs']
mean_obs_obs = data['mean_obs_obs']
std_obs_obs = data['std_obs_obs']
lon_tilenum = data['lon_tilenum']
lat_tilenum = data['lat_tilenum']

std_ascat_obs = np.empty(n_tile)
std_ascat_obs.fill(np.nan)
num_obs_tile = np.empty(n_tile)
num_obs_tile.fill(np.nan)

for i in range(len(unique_tilenum)):
    std_ascat_obs[int(unique_tilenum[i])] = std_obs_obs[i]
    num_obs_tile[int(unique_tilenum[i])] = num_obs[i]

ob_array7 = np.empty([n_tile, 3])
ob_array7.fill(np.nan)
ob_array7[:, 1] = lon
ob_array7[:, 2] = lat
ob_array7[:, 0] = std_ascat_obs

plot_global_tight(ob_array7,False,'ASCAT: StdDev of surface SM','($m^3 \, m^{-3}$)', 0, 0.1, cmap='Blues')


ob_array7 = np.empty([len(lon_tilenum), 3])
ob_array7[:, 1] = lon_tilenum
ob_array7[:, 2] = lat_tilenum
ob_array7[:, 0] = std_obs_obs

plot_global_tight(ob_array7,False,'ASCAT: StdDev of surface SM','($m^3 \, m^{-3}$)', 0, 0.1, cmap='Blues')


In [ ]:

# Load the calculated values from the file
data = np.load('../test_data/OLv7_M36_ascat_fp_precip_SMAP_L4_SM_gph_stats.npz' , allow_pickle=True)
mean_sm_surface = data['mean_sm_surface']
std_sm_surface = data['std_sm_surface']
mean_sm_rootzone = data['mean_sm_rootzone']
std_sm_rootzone = data['std_sm_rootzone']
mean_sm_profile = data['mean_sm_profile']
std_sm_profile = data['std_sm_profile']
mean_precipitation_total_surface_flux = data['mean_precipitation_total_surface_flux']
mean_vegetation_greenness_fraction = data['mean_vegetation_greenness_fraction']
max_vegetation_greenness_fraction = data['max_vegetation_greenness_fraction']
mean_leaf_area_index = data['mean_leaf_area_index']
max_leaf_area_index = data['max_leaf_area_index']

ob_array7 = np.empty([n_tile, 3])
ob_array7.fill(np.nan)
ob_array7[:, 1] = lon
ob_array7[:, 2] = lat
ob_array7[:, 0] = std_sm_surface

plot_global_tight(ob_array7,False,'OLv7_M36: StdDev of surface SM','($m^3 \, m^{-3}$)', 0, 0.1, cmap='Blues')



In [ ]:
# Open file ../test_data/clsm/soil_param.dat and read in the 14th column
with open('../test_data/clsm/soil_param.dat', 'r') as file:
    data = file.readlines()

# Extract the 14th column
column_14 = [float(line.split()[13]) for line in data]

# Convert the list to a numpy array
sand_top = np.array(column_14)

# Print the first 10 values
print(sand_top[:10])

# Plot the data
ob_array8 = np.empty([n_tile, 3])
ob_array8.fill(np.nan)
ob_array8[:, 1] = lon
ob_array8[:, 2] = lat
ob_array8[:, 0] = sand_top

plot_global_tight(ob_array8,False,'CLSM: Sand fraction at top soil layer','(-)', 0, 100, cmap='cividis')

In [ ]:
# Open file ../test_data/clsm/CLM4.5_veg_typs_fracs and read in the 3rd column called CLM-C_pt1

with open('../test_data/clsm/CLM4.5_veg_typs_fracs', 'r') as file:
    data = file.readlines()

# Extract the 3rd column
column_3 = [float(line.split()[2]) for line in data]

# Convert the list to a numpy array
clm_c_pt1 = np.array(column_3)

# Print all the unique values
print(np.unique(clm_c_pt1))

# Plot the data

ob_array9 = np.empty([n_tile, 3])
ob_array9.fill(np.nan)
ob_array9[:, 1] = lon
ob_array9[:, 2] = lat
ob_array9[:, 0] = clm_c_pt1
# ob_array9[:, 0] = 1 where clm_c_pt1 = 10, 0 otherwise
ob_array9[:, 0] = np.where(clm_c_pt1 == 10, 1, 0)

plot_global_tight(ob_array9,False,'CLSM: CLM-C_pt1','(-)', 0, 1, cmap='cividis')


In [ ]:
delta_anomaly = ob_array3[:,0]


plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly

# Plot the data
plot_global_tight(plot_array,True,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

plot_array[:, 0] = std_sm_surface
plot_global_tight(plot_array,True,'OLv7_M36: StdDev of surface SM','($m^3 \, m^{-3}$)', 0, 0.1, cmap='Blues')

fig, ax = plt.subplots()
ax.scatter(std_sm_surface, delta_anomaly, s=2)
xlabel = 'OLv7_M36: StdDev of surface SM'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(std_sm_surface, delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()

# Plot a probability density function of the two variables
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[(plot_array[:, 0] > 0.01) & (plot_array[:, 0] < 0.02)], bins=50, alpha=0.5, label='> 0.01 & < 0.02')
plt.hist(delta_values[(plot_array[:, 0] <= 0.01) | (plot_array[:, 0] >= 0.02)], bins=50, alpha=0.5, label='<=0.01 | >=0.02')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()


In [ ]:

plot_array[:, 0] = std_ascat_obs
plot_global_tight(plot_array,True,'DAv7_M36: StdDev of scaled ASCAT obs','($m^3 \, m^{-3}$)', 0, 0.1, cmap='Blues')

fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'DAv7_M36: StdDev of scaled ASCAT obs'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()

# Plot a probability density function of the two variables
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[(plot_array[:, 0] > 0.01) & (plot_array[:, 0] < 0.02)], bins=50, alpha=0.5, label='>0.01 & <0.02')
plt.hist(delta_values[(plot_array[:, 0] <= 0.01) | (plot_array[:, 0] >= 0.02)], bins=50, alpha=0.5, label='<=0.01 or >=0.02')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array[:, 0] = mean_sm_surface
plot_global_tight(plot_array,True,'OLv7_M36: mean_sm_surface','($m^3 \, m^{-3}$)', 0, 0.5, cmap='Blues')

fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'OLv7_M36: mean_sm_surface'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()

# Plot a probability density function of the two variables
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[(plot_array[:, 0] > 0.05) & (plot_array[:, 0] < 0.08)], bins=50, alpha=0.5, label='>0.05 & <0.08')
plt.hist(delta_values[(plot_array[:, 0] <= 0.05) | (plot_array[:, 0] >= 0.08)], bins=50, alpha=0.5, label='<=0.05 or >=0.08')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array[:, 0] = mean_precipitation_total_surface_flux * 86400
plot_global_tight(plot_array,True,'OLv7_M36: mean_precipitation_total_surface_flux','($kg \, m^{-2} \, d^{-1}$)', 0, 5, cmap='Greens')

fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'OLv7_M36: mean_precipitation_total_surface_flux'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()



# Plot a probability density function of the two variables
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[plot_array[:, 0] < 0.5], bins=50, alpha=0.5, label='< 0.5')
plt.hist(delta_values[plot_array[:, 0] >= 0.5], bins=50, alpha=0.5, label='>= 0.5')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array[:, 0] = max_leaf_area_index
plot_global_tight(plot_array,True,'OLv7_M36: max_leaf_area_index','($m^2 \, m^{-2}$)', 0, 5, cmap='Greens')

fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'OLv7_M36: max_leaf_area_index'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()


# Plot a probability density function of the two variables
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[plot_array[:, 0] < 0.2], bins=50, alpha=0.5, label='< 0.2')
plt.hist(delta_values[plot_array[:, 0] >= 0.2], bins=50, alpha=0.5, label='>= 0.2')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array[:, 0] = mean_leaf_area_index
plot_global_tight(plot_array,True,'OLv7_M36: mean_leaf_area_index','($m^2 \, m^{-2}$)', 0, 5, cmap='Greens')

fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'OLv7_M36: mean_leaf_area_index'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array[:, 0] = sand_top
plot_global_tight(plot_array,True,'OLv7_M36: sand_top','(%)', 0, 100, cmap='Greys')

fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'OLv7_M36: sand_top'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array[:, 0] = clm_c_pt1
plot_array[:, 0] = np.where(clm_c_pt1 == 10, 1, 0)
plot_global_tight(plot_array,True,'OLv7_M36: clm_c_pt1 = 10','(Shrubs)', 0, 1, cmap='Greys')

plot_array[:, 0] = clm_c_pt1
fig, ax = plt.subplots()
ax.scatter(plot_array[:, 0], delta_anomaly, s=2)
xlabel = 'OLv7_M36: clm_c_pt1'
ax.set_xlabel(xlabel)
ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True) 
x_limits = ax.get_xlim()  # Get x limits
y_limits = ax.get_ylim()  # Get y limits
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_scatter.png'
# Save the figure
plt.savefig(filename, dpi=300) 
plt.show()

fig, ax = plt.subplots()
hb = ax.hexbin(plot_array[:, 0], delta_anomaly, gridsize=40, cmap='Reds', mincnt=1)
# ax.set_xlabel('OLv7_M36: StdDev of surface SM')
# ax.set_ylabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
ax.grid(True)
ax.set_xlim(x_limits)  # Set x limits
ax.set_ylim(y_limits)  # Set y limits#
# cb = fig.colorbar(hb, ax=ax)
# cb.set_label('Count')
# Construct the filename
filename = xlabel.replace(' ', '_').replace(':', '') + '_hexbin.png'
# Save the figure
plt.savefig(filename, dpi=300)
plt.show()

# Plot a probability density function of the two variables
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[plot_array[:, 0] == 10], bins=50, alpha=0.5, label='== 10')
plt.hist(delta_values[plot_array[:, 0] != 10], bins=50, alpha=0.5, label='!= 10')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
# Load the calculated values from the file
data = np.load('../test_data/AMS_24/DAv7_M36_ASCAT_type_2_fp_precip_obsfcstana_stats.npz', allow_pickle=True)

# list the variables in the file
print(data.files)

unique_tilenum_fp = data['unique_tilenum']
num_unique_tilenum = len(unique_tilenum_fp)
lon_tilenum = data['lon_tilenum']
lat_tilenum = data['lat_tilenum']
num_obs = data['num_obs']
std_obs_minus_fcst_fp = data['std_obs_minus_fcst']
std_obs_minus_ana_fp = data['std_obs_minus_ana']


obarray = np.empty([num_unique_tilenum, 3])
obarray[:, 1] = lon_tilenum
obarray[:, 2] = lat_tilenum

obarray[:, 0] = num_obs    
plot_global_tight(obarray,False,'ASC_FP: Number of ASCAT Obs Assimilated','Total', 0, 6000, cmap='Greens')

In [ ]:
# Working with the official ASCAT SM mask
# Load the mask netcdf file

ds = xr.open_dataset('../test_data/clsm/ASCAT_mask.nc')
asc_lon = ds['lon']
asc_lat = ds['lat']
asc_mask = ds['mask']

print('len(asc_mask) = ', len(asc_mask))

# Plot the mask
plot_array = np.empty([len(asc_lon), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = asc_lon
plot_array[:, 2] = asc_lat
plot_array[:, 0] = asc_mask

plot_global_tight(plot_array,False,'ASCAT SM mask','(-)', 0, 1, cmap='Greys')


In [ ]:
# Load alternative mask file
ds = xr.open_dataset('../test_data/clsm/subsurface_scattering_ASCAT_ERA5_Land.nc')

cold_mask = ds['cold_mask']
wet_mask = ds['wet_mask']
veg_mask = ds['veg_mask']
subsurface_mask = ds['subsurface_mask']
lon_test = ds['lon']
lat_test = ds['lat']

# Print the first 10 values of the subsurface mask, lat and lon
print(subsurface_mask[:10].values)
print(lat_test[:10].values)
print(lon_test[:10].values)

# Find and print the first 10 values of subsurface mask, lat and lon where subsurface mask is 1
mask = subsurface_mask == 1
print(subsurface_mask[mask][:10].values)
print(lat_test[mask][:10].values)
print(lon_test[mask][:10].values)

# Scatter plot of lon and lat
plt.figure()
plt.scatter(lon_test[mask], lat_test[mask])
plt.xlabel('lon')
plt.ylabel('lat')
plt.show()



In [ ]:


plot_array = np.empty([len(lon_test), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon_test
plot_array[:, 2] = lat_test

test_subsurface_mask = subsurface_mask.values
# test_subsurface_mask[test_subsurface_mask == 0] = 2

plot_array[:, 0] = test_subsurface_mask
plot_global_tight(plot_array,False,'ASCAT subsurface mask','(-)', 0, 2, cmap='Greys')

plot_array[:, 0] = veg_mask
plot_global_tight(plot_array,False,'ASCAT veg mask','(-)', 0, 1, cmap='Greys')

plot_array[:, 0] = wet_mask
plot_global_tight(plot_array,False,'ASCAT wet mask','(-)', 0, 1, cmap='Greys')


In [ ]:

# Make new mask that combines the subsurface, wet and veg masks
combined_mask = np.zeros(len(subsurface_mask))
combined_mask[(subsurface_mask == 1) | (wet_mask == 1) | (veg_mask == 1)] = 1

plot_array[:, 0] = combined_mask
plot_global_tight(plot_array,True,'ASCAT combined mask','(-)', 0, 1, cmap='Greys')


In [ ]:
mask = (subsurface_mask == 1)
asc_lon2 = asc_lon.values[mask]
asc_lat2 = asc_lat.values[mask]
subsurface_mask2 = subsurface_mask.values[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(asc_lon2))
closest_m36_lats = np.empty(len(asc_lat2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))
print(m36_lons.ndim)
print(m36_lats.ndim)

for i in range(len(asc_lon2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - asc_lon2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - asc_lat2[i]))]

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

print(len(closest_m36_lons))
print(n_unique_pairs)

# Find the smcf values for each unique pair
unique_subsurface = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_subsurface[i] = stats.mode(subsurface_mask2[mask])[0][0]

test_array = np.column_stack((unique_subsurface, unique_pairs[:, 0], unique_pairs[:, 1]))

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(test_array, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
subsurface_tile = df3[['value2', 'lon', 'lat']].values

plot_array[:, 0] = subsurface_mask
plot_global_tight(plot_array,True,'ASCAT subsurface mask','(-)', 0, 1, cmap='Greys')

print(len(subsurface_tile))
plot_global_tight(subsurface_tile,False,'M36 subsurface_mask','QC', 0, 1, cmap='viridis')


In [ ]:

# Plot a probability density function of the two variables
print(len(num_obs_tile))

N_a = len(delta_values)
N_a_delta = len(delta_values[~np.isnan(delta_values)])
mean_a = np.nanmean(delta_values)
N_a_pos = len(delta_values[delta_values > 0])
Nobs_a = np.nansum(num_obs_tile)

N_1 = len(delta_values[subsurface_tile[:, 0] == 1]) 
N_1_delta = len(delta_values[(subsurface_tile[:, 0] == 1) & ~np.isnan(delta_values)])
mean_1 = np.nanmean(delta_values[subsurface_tile[:, 0] == 1])
N_1_pos = len(delta_values[(subsurface_tile[:, 0] == 1) & (delta_values > 0)])
Nobs_1 = np.nansum(num_obs_tile[subsurface_tile[:, 0] == 1])

N_0 = len(delta_values[subsurface_tile[:, 0] != 1])
N_0_delta = len(delta_values[(subsurface_tile[:, 0] != 1) & ~np.isnan(delta_values)])
mean_0 = np.nanmean(delta_values[subsurface_tile[:, 0] != 1])
N_0_pos = len(delta_values[(subsurface_tile[:, 0] != 1) & (delta_values > 0)])
Nobs_0 = np.nansum(num_obs_tile[subsurface_tile[:, 0] != 1])

xlabel = 'subsurface_mask'
plt.figure()
plt.title(f'{xlabel}\n All N_tile = {N_a}, N obs = {Nobs_a:.0f}, N IVs tiles = {N_a_delta}, Mean = {mean_a:.3f} ({(N_a_pos/N_a_delta)*100:.0f}% +ve)\n' 
          f'Masked N_tile = {N_1} ({(N_1/N_a)*100:.0f}%), N obs = {Nobs_1:.0f} ({(Nobs_1/Nobs_a)*100:.0f}%), N IVs tiles = {N_1_delta}({(N_1_delta/N_a_delta)*100:.0f}%), Mean = {mean_1:.3f} ({(N_1_pos/N_1_delta)*100:.0f}% +ve)\n'
          f'Unmasked N_tile = {N_0} ({(N_0/N_a)*100:.0f}%), N obs = {Nobs_0:.0f} ({(Nobs_0/Nobs_a)*100:.0f}%), N IVs tiles = {N_0_delta}({(N_0_delta/N_a_delta)*100:.0f}%), Mean = {mean_0:.3f} ({(N_0_pos/N_0_delta)*100:.0f}% +ve)')
plt.hist(delta_values[subsurface_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[subsurface_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly

# Plot the data
plot_global_tight(plot_array,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

masked_delta = np.copy(delta_anomaly)
masked_delta[subsurface_tile[:, 0] == 1] = np.nan

plot_array[:, 0] = masked_delta

# Plot the data
plot_global_tight(plot_array,False,'Surface Soil Moisture Skill (Δ anomaly R): with subsurface_mask applied','(-)', -0.4, 0.4)


In [ ]:
mask = (asc_mask == 1)
asc_lon2 = asc_lon.values[mask]
asc_lat2 = asc_lat.values[mask]
asc_mask2 = asc_mask.values[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(asc_lon2))
closest_m36_lats = np.empty(len(asc_lat2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))
print(m36_lons.ndim)
print(m36_lats.ndim)

for i in range(len(asc_lon2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - asc_lon2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - asc_lat2[i]))]

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

print(len(closest_m36_lons))
print(n_unique_pairs)

# Find the smcf values for each unique pair
unique_asc_mask = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_asc_mask[i] = stats.mode(asc_mask2[mask])[0][0]

test_array = np.column_stack((unique_asc_mask, unique_pairs[:, 0], unique_pairs[:, 1]))

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(test_array, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
asc_mask_tile = df3[['value2', 'lon', 'lat']].values

# Plot the mask
plot_array = np.empty([len(asc_lon), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = asc_lon
plot_array[:, 2] = asc_lat
plot_array[:, 0] = asc_mask

plot_global_tight(plot_array,True,'ASCAT SM mask','(-)', 0, 1, cmap='Greys')

print(len(asc_mask_tile))
plot_global_tight(asc_mask_tile,False,'M36 ASCAT_mask','QC', 0, 1, cmap='viridis')

# Plot a probability density function of the two variables
xlabel = 'ASCAT_mask'
plt.figure()
plt.title(xlabel)
plt.hist(delta_values[asc_mask_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[asc_mask_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

N_a = len(delta_values)
N_a_delta = len(delta_values[~np.isnan(delta_values)])
mean_a = np.nanmean(delta_values)
N_a_pos = len(delta_values[delta_values > 0])
Nobs_a = np.nansum(num_obs_tile)

N_1 = len(delta_values[asc_mask_tile[:, 0] == 1]) 
N_1_delta = len(delta_values[(asc_mask_tile[:, 0] == 1) & ~np.isnan(delta_values)])
mean_1 = np.nanmean(delta_values[asc_mask_tile[:, 0] == 1])
N_1_pos = len(delta_values[(asc_mask_tile[:, 0] == 1) & (delta_values > 0)])
Nobs_1 = np.nansum(num_obs_tile[asc_mask_tile[:, 0] == 1])

N_0 = len(delta_values[asc_mask_tile[:, 0] != 1])
N_0_delta = len(delta_values[(asc_mask_tile[:, 0] != 1) & ~np.isnan(delta_values)])
mean_0 = np.nanmean(delta_values[asc_mask_tile[:, 0] != 1])
N_0_pos = len(delta_values[(asc_mask_tile[:, 0] != 1) & (delta_values > 0)])
Nobs_0 = np.nansum(num_obs_tile[asc_mask_tile[:, 0] != 1])

xlabel = 'asc_mask_tile'
plt.figure()
plt.title(f'{xlabel}\n All N_tile = {N_a}, N obs = {Nobs_a:.0f}, N IVs tiles = {N_a_delta}, Mean = {mean_a:.3f} ({(N_a_pos/N_a_delta)*100:.0f}% +ve)\n' 
          f'Masked N_tile = {N_1} ({(N_1/N_a)*100:.0f}%), N obs = {Nobs_1:.0f} ({(Nobs_1/Nobs_a)*100:.0f}%), N IVs tiles = {N_1_delta}({(N_1_delta/N_a_delta)*100:.0f}%), Mean = {mean_1:.3f} ({(N_1_pos/N_1_delta)*100:.0f}% +ve)\n'
          f'Unmasked N_tile = {N_0} ({(N_0/N_a)*100:.0f}%), N obs = {Nobs_0:.0f} ({(Nobs_0/Nobs_a)*100:.0f}%), N IVs tiles = {N_0_delta}({(N_0_delta/N_a_delta)*100:.0f}%), Mean = {mean_0:.3f} ({(N_0_pos/N_0_delta)*100:.0f}% +ve)')
plt.hist(delta_values[asc_mask_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[asc_mask_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()



In [ ]:

plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly
# Plot the data
plot_global_tight(plot_array,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

masked_delta = np.copy(delta_anomaly)
masked_delta[asc_mask_tile[:, 0] == 1] = np.nan

plot_array[:, 0] = masked_delta

# Plot the data
plot_global_tight(plot_array,False,'Surface Soil Moisture Skill (Δ anomaly R): with asc_mask_tile applied','(-)', -0.4, 0.4)

In [ ]:
mask = (combined_mask == 1)
asc_lon2 = asc_lon.values[mask]
asc_lat2 = asc_lat.values[mask]
combined_mask2 = combined_mask[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(asc_lon2))
closest_m36_lats = np.empty(len(asc_lat2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))
print(m36_lons.ndim)
print(m36_lats.ndim)

for i in range(len(asc_lon2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - asc_lon2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - asc_lat2[i]))]

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

print(len(closest_m36_lons))
print(n_unique_pairs)

# Find the smcf values for each unique pair
unique_combined_mask = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_combined_mask[i] = stats.mode(combined_mask2[mask])[0][0]

test_array = np.column_stack((unique_combined_mask, unique_pairs[:, 0], unique_pairs[:, 1]))

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(test_array, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
combined_mask_tile = df3[['value2', 'lon', 'lat']].values

# Plot the mask
plot_array = np.empty([len(asc_lon), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = asc_lon
plot_array[:, 2] = asc_lat
plot_array[:, 0] = combined_mask
plot_global_tight(plot_array,True,'ASCAT combined mask','(-)', 0, 1, cmap='Greys')

print(len(combined_mask_tile))
plot_global_tight(combined_mask_tile,False,'M36 combined_mask','QC', 0, 1, cmap='viridis')



In [ ]:
print(delta_values[combined_mask_tile[:, 0] == 1])

In [ ]:

# Plot a probability density function of the two variables
xlabel = 'combined_mask'
plt.figure()
plt.title(xlabel)
plt.hist(delta_values, bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[combined_mask_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[combined_mask_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

N_a = len(delta_values)
N_a_delta = len(delta_values[~np.isnan(delta_values)])
mean_a = np.nanmean(delta_values)
N_a_pos = len(delta_values[delta_values > 0])
Nobs_a = np.nansum(num_obs_tile)

N_1 = len(delta_values[combined_mask_tile[:, 0] == 1]) 
N_1_delta = len(delta_values[(combined_mask_tile[:, 0] == 1) & ~np.isnan(delta_values)])
mean_1 = np.nanmean(delta_values[combined_mask_tile[:, 0] == 1])
N_1_pos = len(delta_values[(combined_mask_tile[:, 0] == 1) & (delta_values > 0)])
Nobs_1 = np.nansum(num_obs_tile[combined_mask_tile[:, 0] == 1])

N_0 = len(delta_values[combined_mask_tile[:, 0] != 1])
N_0_delta = len(delta_values[(combined_mask_tile[:, 0] != 1) & ~np.isnan(delta_values)])
mean_0 = np.nanmean(delta_values[combined_mask_tile[:, 0] != 1])
N_0_pos = len(delta_values[(combined_mask_tile[:, 0] != 1) & (delta_values > 0)])
Nobs_0 = np.nansum(num_obs_tile[combined_mask_tile[:, 0] != 1])

xlabel = 'combined_mask_tile'
plt.figure()
plt.title(f'{xlabel}\n All N_tile = {N_a}, N obs = {Nobs_a:.0f}, N IVs tiles = {N_a_delta}, Mean = {mean_a:.3f} ({(N_a_pos/N_a_delta)*100:.0f}% +ve)\n' 
          f'Masked N_tile = {N_1} ({(N_1/N_a)*100:.0f}%), N obs = {Nobs_1:.0f} ({(Nobs_1/Nobs_a)*100:.0f}%), N IVs tiles = {N_1_delta}({(N_1_delta/N_a_delta)*100:.0f}%), Mean = {mean_1:.3f} ({(N_1_pos/N_1_delta)*100:.0f}% +ve)\n'
          f'Unmasked N_tile = {N_0} ({(N_0/N_a)*100:.0f}%), N obs = {Nobs_0:.0f} ({(Nobs_0/Nobs_a)*100:.0f}%), N IVs tiles = {N_0_delta}({(N_0_delta/N_a_delta)*100:.0f}%), Mean = {mean_0:.3f} ({(N_0_pos/N_0_delta)*100:.0f}% +ve)')
plt.hist(delta_values[combined_mask_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[combined_mask_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly
# Plot the data
plot_global_tight(plot_array,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

masked_delta = np.copy(delta_anomaly)
masked_delta[combined_mask_tile[:, 0] == 1] = np.nan

plot_array[:, 0] = masked_delta

# Plot the data
plot_global_tight(plot_array,False,'Surface Soil Moisture Skill (Δ anomaly R): with asc_mask_tile applied','(-)', -0.4, 0.4)

In [ ]:
mask = (veg_mask == 1)
asc_lon2 = asc_lon.values[mask]
asc_lat2 = asc_lat.values[mask]
veg_mask2 = veg_mask.values[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(asc_lon2))
closest_m36_lats = np.empty(len(asc_lat2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))
print(m36_lons.ndim)
print(m36_lats.ndim)

for i in range(len(asc_lon2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - asc_lon2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - asc_lat2[i]))]

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

print(len(closest_m36_lons))
print(n_unique_pairs)

# Find the smcf values for each unique pair
unique_veg = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_veg[i] = stats.mode(veg_mask2[mask])[0][0]

test_array = np.column_stack((unique_veg, unique_pairs[:, 0], unique_pairs[:, 1]))

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(test_array, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
veg_tile = df3[['value2', 'lon', 'lat']].values

# Plot the mask
plot_array = np.empty([len(asc_lon), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = asc_lon
plot_array[:, 2] = asc_lat
plot_array[:, 0] = veg_mask
plot_global_tight(plot_array,True,'ASCAT veg mask','(-)', 0, 1, cmap='Greys')

print(len(veg_tile))
plot_global_tight(veg_tile,False,'M36 veg_mask','QC', 0, 1, cmap='viridis')

In [ ]:
# Plot a probability density function of the two variables
print(len(num_obs_tile))

N_a = len(delta_values)
N_a_delta = len(delta_values[~np.isnan(delta_values)])
mean_a = np.nanmean(delta_values)
N_a_pos = len(delta_values[delta_values > 0])
Nobs_a = np.nansum(num_obs_tile)

N_1 = len(delta_values[veg_tile[:, 0] == 1]) 
N_1_delta = len(delta_values[(veg_tile[:, 0] == 1) & ~np.isnan(delta_values)])
mean_1 = np.nanmean(delta_values[veg_tile[:, 0] == 1])
N_1_pos = len(delta_values[(veg_tile[:, 0] == 1) & (delta_values > 0)])
Nobs_1 = np.nansum(num_obs_tile[veg_tile[:, 0] == 1])

N_0 = len(delta_values[veg_tile[:, 0] != 1])
N_0_delta = len(delta_values[(veg_tile[:, 0] != 1) & ~np.isnan(delta_values)])
mean_0 = np.nanmean(delta_values[veg_tile[:, 0] != 1])
N_0_pos = len(delta_values[(veg_tile[:, 0] != 1) & (delta_values > 0)])
Nobs_0 = np.nansum(num_obs_tile[veg_tile[:, 0] != 1])

xlabel = 'veg_mask'
plt.figure()
plt.title(f'{xlabel}\n All N_tile = {N_a}, N obs = {Nobs_a:.0f}, N IVs tiles = {N_a_delta}, Mean = {mean_a:.3f} ({(N_a_pos/N_a_delta)*100:.0f}% +ve)\n' 
          f'Masked N_tile = {N_1} ({(N_1/N_a)*100:.0f}%), N obs = {Nobs_1:.0f} ({(Nobs_1/Nobs_a)*100:.0f}%), N IVs tiles = {N_1_delta}({(N_1_delta/N_a_delta)*100:.0f}%), Mean = {mean_1:.3f} ({(N_1_pos/N_1_delta)*100:.0f}% +ve)\n'
          f'Unmasked N_tile = {N_0} ({(N_0/N_a)*100:.0f}%), N obs = {Nobs_0:.0f} ({(Nobs_0/Nobs_a)*100:.0f}%), N IVs tiles = {N_0_delta}({(N_0_delta/N_a_delta)*100:.0f}%), Mean = {mean_0:.3f} ({(N_0_pos/N_0_delta)*100:.0f}% +ve)')
plt.hist(delta_values[veg_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[veg_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly
# Plot the data
plot_global_tight(plot_array,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

masked_delta = np.copy(delta_anomaly)
masked_delta[veg_tile[:, 0] == 1] = np.nan

plot_array[:, 0] = masked_delta

# Plot the data
plot_global_tight(plot_array,False,'Surface Soil Moisture Skill (Δ anomaly R): with veg_tile applied','(-)', -0.4, 0.4)

In [ ]:
mask = (wet_mask == 1)
asc_lon2 = asc_lon.values[mask]
asc_lat2 = asc_lat.values[mask]
wet_mask2 = wet_mask.values[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(asc_lon2))
closest_m36_lats = np.empty(len(asc_lat2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))
print(m36_lons.ndim)
print(m36_lats.ndim)

for i in range(len(asc_lon2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - asc_lon2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - asc_lat2[i]))]

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

print(len(closest_m36_lons))
print(n_unique_pairs)

# Find the smcf values for each unique pair
unique_wet = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_wet[i] = stats.mode(wet_mask2[mask])[0][0]

test_array = np.column_stack((unique_wet, unique_pairs[:, 0], unique_pairs[:, 1]))

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(test_array, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
wet_tile = df3[['value2', 'lon', 'lat']].values

# Plot the mask
plot_array = np.empty([len(asc_lon), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = asc_lon
plot_array[:, 2] = asc_lat
plot_array[:, 0] = wet_mask
plot_global_tight(plot_array,True,'ASCAT wet mask','(-)', 0, 1, cmap='Greys')

print(len(wet_tile))
plot_global_tight(wet_tile,False,'M36 wet_mask','QC', 0, 1, cmap='viridis')

In [ ]:
# Plot a probability density function of the two variables
print(len(num_obs_tile))

N_a = len(delta_values)
N_a_delta = len(delta_values[~np.isnan(delta_values)])
mean_a = np.nanmean(delta_values)
N_a_pos = len(delta_values[delta_values > 0])
Nobs_a = np.nansum(num_obs_tile)

N_1 = len(delta_values[wet_tile[:, 0] == 1]) 
N_1_delta = len(delta_values[(wet_tile[:, 0] == 1) & ~np.isnan(delta_values)])
mean_1 = np.nanmean(delta_values[wet_tile[:, 0] == 1])
N_1_pos = len(delta_values[(wet_tile[:, 0] == 1) & (delta_values > 0)])
Nobs_1 = np.nansum(num_obs_tile[wet_tile[:, 0] == 1])

N_0 = len(delta_values[wet_tile[:, 0] != 1])
N_0_delta = len(delta_values[(wet_tile[:, 0] != 1) & ~np.isnan(delta_values)])
mean_0 = np.nanmean(delta_values[wet_tile[:, 0] != 1])
N_0_pos = len(delta_values[(wet_tile[:, 0] != 1) & (delta_values > 0)])
Nobs_0 = np.nansum(num_obs_tile[wet_tile[:, 0] != 1])

xlabel = 'wet_mask'
plt.figure()
plt.title(f'{xlabel}\n All N_tile = {N_a}, N obs = {Nobs_a:.0f}, N IVs tiles = {N_a_delta}, Mean = {mean_a:.3f} ({(N_a_pos/N_a_delta)*100:.0f}% +ve)\n' 
          f'Masked N_tile = {N_1} ({(N_1/N_a)*100:.0f}%), N obs = {Nobs_1:.0f} ({(Nobs_1/Nobs_a)*100:.0f}%), N IVs tiles = {N_1_delta}({(N_1_delta/N_a_delta)*100:.0f}%), Mean = {mean_1:.3f} ({(N_1_pos/N_1_delta)*100:.0f}% +ve)\n'
          f'Unmasked N_tile = {N_0} ({(N_0/N_a)*100:.0f}%), N obs = {Nobs_0:.0f} ({(Nobs_0/Nobs_a)*100:.0f}%), N IVs tiles = {N_0_delta}({(N_0_delta/N_a_delta)*100:.0f}%), Mean = {mean_0:.3f} ({(N_0_pos/N_0_delta)*100:.0f}% +ve)')
plt.hist(delta_values[wet_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[wet_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly
# Plot the data
plot_global_tight(plot_array,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

masked_delta = np.copy(delta_anomaly)
masked_delta[wet_tile[:, 0] == 1] = np.nan

plot_array[:, 0] = masked_delta

# Plot the data
plot_global_tight(plot_array,False,'Surface Soil Moisture Skill (Δ anomaly R): with wet_tile applied','(-)', -0.4, 0.4)

In [ ]:
# Make new mask that combines the subsurface and wet masks
combi_mask = np.zeros(len(subsurface_mask))
combi_mask[(subsurface_mask == 1) | (wet_mask == 1)] = 1

mask = (combi_mask == 1)
asc_lon2 = asc_lon.values[mask]
asc_lat2 = asc_lat.values[mask]
combi_mask2 = combi_mask[mask]

# Find the closest m36_lons value to each of the qc_lons value
closest_m36_lons = np.empty(len(asc_lon2))
closest_m36_lats = np.empty(len(asc_lat2))

print('len(m36_lons) = ', len(m36_lons))
print('len(m36_lats) = ', len(m36_lats))
print(m36_lons.ndim)
print(m36_lats.ndim)

for i in range(len(asc_lon2)):
    closest_m36_lons[i] = m36_lons[np.argmin(np.abs(m36_lons - asc_lon2[i]))]
    closest_m36_lats[i] = m36_lats[np.argmin(np.abs(m36_lats - asc_lat2[i]))]

#Find the unique pairs of closest_m36_lons and closest_m36_lats
unique_pairs = np.unique(np.column_stack((closest_m36_lons, closest_m36_lats)), axis=0)

# Find the number of unique pairs
n_unique_pairs = len(unique_pairs)

print(len(closest_m36_lons))
print(n_unique_pairs)

# Find the smcf values for each unique pair
unique_combi = np.empty(n_unique_pairs)
for i in range(n_unique_pairs):
    mask = (closest_m36_lons == unique_pairs[i, 0]) & (closest_m36_lats == unique_pairs[i, 1])
    unique_combi[i] = stats.mode(combi_mask2[mask])[0][0]

test_array = np.column_stack((unique_combi, unique_pairs[:, 0], unique_pairs[:, 1]))

# Convert ob_array and ob_array2 to pandas DataFrames
df1 = pd.DataFrame(ob_array, columns=['value', 'lon', 'lat'])
df2 = pd.DataFrame(test_array, columns=['value2', 'lon2', 'lat2'])

# Round the lat and lon columns to 3 decimal places
df1[['lon', 'lat']] = df1[['lon', 'lat']].round(3)
df2[['lon2', 'lat2']] = df2[['lon2', 'lat2']].round(3)

# Merge the two DataFrames on the rounded lat and lon columns
df3 = pd.merge(df1, df2, left_on=['lon', 'lat'], right_on=['lon2', 'lat2'], how='left')

# Convert the merged DataFrame back to a numpy array
combi_tile = df3[['value2', 'lon', 'lat']].values

# Plot the mask
plot_array = np.empty([len(asc_lon), 3])
plot_array.fill(np.nan)
plot_array[:, 1] = asc_lon
plot_array[:, 2] = asc_lat
plot_array[:, 0] = combi_mask
plot_global_tight(plot_array,True,'ASCAT SS and wet mask','(-)', 0, 1, cmap='Greys')

print(len(combi_tile))
plot_global_tight(combi_tile,False,'M36 combi_mask (SS and wet)','QC', 0, 1, cmap='viridis')

In [ ]:
# Plot a probability density function of the two variables
print(len(num_obs_tile))

N_a = len(delta_values)
N_a_delta = len(delta_values[~np.isnan(delta_values)])
mean_a = np.nanmean(delta_values)
N_a_pos = len(delta_values[delta_values > 0])
Nobs_a = np.nansum(num_obs_tile)

N_1 = len(delta_values[combi_tile[:, 0] == 1]) 
N_1_delta = len(delta_values[(combi_tile[:, 0] == 1) & ~np.isnan(delta_values)])
mean_1 = np.nanmean(delta_values[combi_tile[:, 0] == 1])
N_1_pos = len(delta_values[(combi_tile[:, 0] == 1) & (delta_values > 0)])
Nobs_1 = np.nansum(num_obs_tile[combi_tile[:, 0] == 1])

N_0 = len(delta_values[combi_tile[:, 0] != 1])
N_0_delta = len(delta_values[(combi_tile[:, 0] != 1) & ~np.isnan(delta_values)])
mean_0 = np.nanmean(delta_values[combi_tile[:, 0] != 1])
N_0_pos = len(delta_values[(combi_tile[:, 0] != 1) & (delta_values > 0)])
Nobs_0 = np.nansum(num_obs_tile[combi_tile[:, 0] != 1])

xlabel = 'combi_mask (SS and wet)'
plt.figure()
plt.title(f'{xlabel}\n All N_tile = {N_a}, N obs = {Nobs_a:.0f}, N IVs tiles = {N_a_delta}, Mean = {mean_a:.3f} ({(N_a_pos/N_a_delta)*100:.0f}% +ve)\n' 
          f'Masked N_tile = {N_1} ({(N_1/N_a)*100:.0f}%), N obs = {Nobs_1:.0f} ({(Nobs_1/Nobs_a)*100:.0f}%), N IVs tiles = {N_1_delta}({(N_1_delta/N_a_delta)*100:.0f}%), Mean = {mean_1:.3f} ({(N_1_pos/N_1_delta)*100:.0f}% +ve)\n'
          f'Unmasked N_tile = {N_0} ({(N_0/N_a)*100:.0f}%), N obs = {Nobs_0:.0f} ({(Nobs_0/Nobs_a)*100:.0f}%), N IVs tiles = {N_0_delta}({(N_0_delta/N_a_delta)*100:.0f}%), Mean = {mean_0:.3f} ({(N_0_pos/N_0_delta)*100:.0f}% +ve)')
plt.hist(delta_values[combi_tile[:, 0] == 1], bins=50, alpha=0.5, label='== 1')
plt.hist(delta_values[combi_tile[:, 0] != 1], bins=50, alpha=0.5, label='!= 1')
plt.xlabel('ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)')
plt.ylabel('Count')
plt.axvline(0, color='grey', linestyle='--')  # Add dashed vertical line at zero
plt.legend(loc='upper right')
filename = xlabel.replace(' ', '_').replace(':', '') + '_pdf.png'
plt.savefig(filename, dpi=300)
plt.show()

plot_array = np.empty([n_tile, 3])
plot_array.fill(np.nan)
plot_array[:, 1] = lon
plot_array[:, 2] = lat
plot_array[:, 0] = delta_anomaly
# Plot the data
plot_global_tight(plot_array,False,'ASC_FP minus CTRL_FP: Surface Soil Moisture Skill (Δ anomaly R)','(-)', -0.4, 0.4)

masked_delta = np.copy(delta_anomaly)
masked_delta[combi_tile[:, 0] == 1] = np.nan

plot_array[:, 0] = masked_delta

# Plot the data
plot_global_tight(plot_array,False,'Surface Soil Moisture Skill (Δ anomaly R): with combi_tile applied','(-)', -0.4, 0.4)